# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770834400869                   -0.52    9.0    168ms
  2   -2.772146835488       -2.88       -1.33    1.0    137ms
  3   -2.772170151442       -4.63       -2.41    1.0    123ms
  4   -2.772170644270       -6.31       -3.11    1.0    112ms
  5   -2.772170722690       -7.11       -4.43    2.0    139ms
  6   -2.772170722854       -9.79       -4.57    1.0    116ms
  7   -2.772170723012       -9.80       -5.56    1.0    120ms
  8   -2.772170723014      -11.77       -5.63    2.0    138ms
  9   -2.772170723015      -12.01       -6.14    1.0    123ms
 10   -2.772170723015   +  -14.01       -6.44    1.0    127ms
 11   -2.772170723015      -13.76       -6.76    1.0    127ms
 12   -2.772170723015      -14.27       -7.64    1.0    569ms
 13   -2.772170723015   +  -15.35       -8.01    2.0    133ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ----

1.7735578746157374

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770741925591                   -0.53    9.0    190ms
  2   -2.772053315436       -2.88       -1.31    1.0    122ms
  3   -2.772083024936       -4.53       -2.56    1.0    112ms
  4   -2.772083374980       -6.46       -3.52    1.0    116ms
  5   -2.772083415411       -7.39       -3.84    2.0    132ms
  6   -2.772083417737       -8.63       -4.98    1.0    118ms
  7   -2.772083417810      -10.14       -5.53    2.0    136ms
  8   -2.772083417810      -12.03       -5.92    2.0    670ms
  9   -2.772083417811      -12.88       -6.49    1.0    138ms
 10   -2.772083417811      -13.71       -7.64    1.0    149ms
 11   -2.772083417811      -14.31       -7.92    2.0    147ms
 12   -2.772083417811   +  -14.40       -8.49    1.0    125ms

Polarizability via ForwardDiff:       1.7725349645551904
Polarizability via finite difference: 1.7735578746157374
